## Data Source

https://api.covid19india.org/  
https://api.covid19india.org/csv/

Total Confirmed Cases - https://public.flourish.studio/visualisation/2917845/ <br>
Total Recovered - https://app.flourish.studio/visualisation/2917909/ <br>
Confirmed vs Recovered - https://app.flourish.studio/visualisation/2917932/ <br>
Confirmed vs Deceased - https://app.flourish.studio/visualisation/2917972/ <br>

## Import libraries

In [2]:
import re
import requests
import json
import math
import csv
import numpy as np
import pandas as pd
import folium
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Daily cases

In [3]:
day_wise = pd.read_csv('https://api.covid19india.org/csv/latest/case_time_series.csv')

# day_wise.to_csv('/home/adi/Desktop/CODS-NLP-ML-Team/covid-19-india-data/data/nation_level_daily.csv', index=False)

day_wise.to_csv('./data/nation_level_daily.csv', index=False)

day_wise.tail()

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
155,03 July,22718,649889,14417,394320,444,18671
156,04 July,24018,673907,14746,409066,611,19282
157,05 July,23942,697849,15829,424895,421,19703
158,06 July,22500,720349,15315,440210,473,20176
159,07 July,23147,743496,16851,457061,479,20655


In [4]:
def timeformat(x):
    d = datetime.strptime(x, '%d %B , %Y')
    x = d.strftime('%Y-%m-%d')
    return x

In [5]:
day_wise["Date"] = day_wise["Date"].apply(lambda x: x+", 2020")

In [6]:
day_wise["Date"] = day_wise["Date"].apply(timeformat)

In [7]:
day_wise

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
0,2020-01-30,1,1,0,0,0,0
1,2020-01-31,0,1,0,0,0,0
2,2020-02-01,0,1,0,0,0,0
3,2020-02-02,1,2,0,0,0,0
4,2020-02-03,1,3,0,0,0,0
...,...,...,...,...,...,...,...
155,2020-07-03,22718,649889,14417,394320,444,18671
156,2020-07-04,24018,673907,14746,409066,611,19282
157,2020-07-05,23942,697849,15829,424895,421,19703
158,2020-07-06,22500,720349,15315,440210,473,20176


In [8]:
no_rows = len(day_wise)-1
perspective_deaths = math.floor(day_wise["Daily Deceased"][no_rows]/day_wise["Daily Confirmed"][no_rows]*100)

In [9]:
str(perspective_deaths)+" in 100 people die due to COVID-19 in India based on the current stats."

'2 in 100 people die due to COVID-19 in India based on the current stats.'

## Latest - State wise 

In [10]:
latest = pd.read_csv('https://api.covid19india.org/csv/latest/state_wise.csv')

# latest.to_csv('/home/adi/Desktop/CODS-NLP-ML-Team/covid-19-india-data/data/state_level_latest.csv', index=False)
latest.to_csv('./data/state_level_latest.csv', index=False)
latest.drop(columns=["Migrated_Other","Delta_Confirmed","Delta_Recovered","Delta_Deaths","State_Notes"],axis=1,inplace=True)
latest.drop(latest.index[0],inplace=True)
latest.drop(latest.index[9],inplace=True)
latest.reset_index(drop=True)

,State,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,State_code
0,Maharashtra,217121,118558,9250,89294,07/07/2020 23:07:39,MH
1,Tamil Nadu,118594,71116,1636,45842,07/07/2020 20:07:40,TN
2,Delhi,102831,74217,3165,25449,07/07/2020 19:47:44,DL
3,Gujarat,37636,26744,1978,8914,07/07/2020 23:26:47,GJ
4,Uttar Pradesh,29968,19627,827,9514,07/07/2020 20:39:34,UP
5,Rajasthan,21577,16583,478,4516,08/07/2020 11:51:27,RJ
6,West Bengal,23837,15790,804,7243,07/07/2020 21:08:28,WB
7,Madhya Pradesh,15627,11768,622,3237,07/07/2020 20:39:41,MP
8,Haryana,18336,13759,279,4298,08/07/2020 15:38:29,HR
9,Karnataka,26815,11100,417,15294,07/07/2020 20:39:43,KA


In [11]:
df = pd.read_html("https://distancelatlong.com/country/india#table table-striped setBorder")

In [12]:
df = df[2]

In [13]:
latest = latest.sort_values('State')

In [14]:
line = pd.DataFrame({"States": "Ladakh", "Latitude": 34.209515, "Longitude":77.615112}, index=[17])
df2 = pd.concat([df.iloc[:16], line, df.iloc[16:]]).reset_index(drop=True)
line1 = pd.DataFrame({"States": "Gujarat", "Latitude": 22.2587, "Longitude":71.1924}, index=[11])
df3 = pd.concat([df2.iloc[:10], line1, df2.iloc[10:]]).reset_index(drop=True)
line2 = pd.DataFrame({"States": "Telangana", "Latitude": 18.1124, "Longitude":79.0193}, index=[32])
df4 = pd.concat([df3.iloc[:31], line2, df3.iloc[31:]]).reset_index(drop=True)
df4.sort_values("States")
df4.reset_index(drop=True)
df = df4

In [15]:
df = df.sort_values("States")
df.drop(columns="States",axis=1,inplace=True)

In [16]:
latest = latest.reset_index(drop=True)
latest = pd.concat([latest,df],axis=1)

In [17]:
latest

,State,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,State_code,Latitude,Longitude
0,Andaman and Nicobar Islands,149,74,0,75,08/07/2020 09:57:39,AN,11.667026,92.735983
1,Andhra Pradesh,22259,11101,264,10894,08/07/2020 14:18:28,AP,14.750429,78.570026
2,Arunachal Pradesh,276,105,2,169,08/07/2020 00:17:28,AR,27.100399,93.616601
3,Assam,13337,8330,21,4983,08/07/2020 12:25:29,AS,26.749981,94.216667
4,Bihar,13274,9541,100,3633,08/07/2020 17:25:28,BR,25.785414,87.479973
5,Chandigarh,492,401,7,84,07/07/2020 15:27:31,CH,30.719997,76.780006
6,Chhattisgarh,3415,2728,14,673,07/07/2020 20:19:48,CT,22.090420,82.159987
7,Dadra and Nagar Haveli and Daman and Diu,442,196,1,242,08/07/2020 16:38:34,DN,20.266578,73.016618
8,Delhi,102831,74217,3165,25449,07/07/2020 19:47:44,DL,28.669993,77.230004
9,Goa,1903,1156,8,739,07/07/2020 20:51:41,GA,15.491997,73.818001


In [18]:
latest["Last_Updated_Time"] = latest["Last_Updated_Time"].apply(lambda x:x.split()[0])

In [19]:
# tests_state_wise = tests_state_wise.reset_index(drop=True)
latest = latest.astype(str)
latest.to_csv("latest.csv",index=False)

In [20]:
india = folium.Map(location = [20.5937,78.9629],zoom_start=4.5)

for date,state,lat,long,total_cases,Death,Recov,Active in zip(list(latest["Last_Updated_Time"]),list(latest['State']),list(latest['Latitude']),list(latest['Longitude']),list(latest['Confirmed']),list(latest['Deaths']),list(latest['Recovered']),list(latest['Active'])):
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(india)
    folium.Marker(location = [lat,long],
                  popup=folium.Popup(('<strong><font color= blue>Date : '+date+'</font></striong><br>' +
                    '<strong><b>State  : '+state+'</strong> <br>' +
                    '<strong><b>Total Cases : '+total_cases+'</striong><br>' +
                    '<strong><font color= red>Deaths : </font>'+Death+'</striong><br>' +
                    '<strong><font color=green>Recoveries : </font>'+Recov+'</striong><br>'+
                    '<strong><b>Active Cases : '+Active+'</striong>'  ),max_width=200)).add_to(india)
india

In [21]:
india.save(outfile="india.html")